# PicoScope: Zero-Crossing Clock Jitter Measurement

In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "data_store"

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
# You need to fix this path when running!
%run "../../../Setup_Scripts/Setup_Generic.ipynb"

### Build Firmware - Internal Clock Output on Pin

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, r"firmwares/fw-120mhz-clkout.hex")

In [ ]:
#Baud is lower so it works from internal oscillator which isn't as precise
target.baud = 38400

### ChipWhisperer Setup - clock from external target only

In [ ]:
import time
scope.clock.clkgen_src = "extclk"
scope.io.hs2 = None

In [ ]:
scope.clock.pll.set_outfreqs(60E6, 60E6, 2, True)

In [ ]:
scope.clock

## PicoScope Capture Setup (Shared for any PicoScope Measurement)

This does intial setup, and can be used for a quick shunt comparison


In [ ]:
import os
picoinstallpath = os.path.normpath(r"C:\Program Files\Pico Technology\PicoScope 7 T&M Early Access")
if picoinstallpath not in os.environ['PATH']:
    print("Adding Pico Install to Path")
    os.environ['PATH'] = picoinstallpath + os.pathsep + os.environ['PATH']
else:
    print("Pico Install Already on Path")

In [ ]:
from picoscope import ps6000
ps = ps6000.PS6000()

In [ ]:
ps.setChannel(channel='A', coupling='AC', VRange=0.1, VOffset=0.0, enabled=True, BWLimited=0, probeAttenuation=1.0)
ps.setChannel(channel='B', coupling='AC', VRange=5.0, VOffset=0.0, enabled=True, BWLimited=0, probeAttenuation=10.0)
ps.setChannel(channel='C', enabled=False)
ps.setChannel(channel='D', enabled=False)

In [ ]:
#Setup timebase
samplerate, maxsamples = ps.setSamplingFrequency(1.25E9, 50E3)
print("Actual = %f GS/s" % (samplerate/1E9))

In [ ]:
ps.setSimpleTrigger('B', 1.5, timeout_ms=1000)

### T-Test of Shunt Measurement (Setup Validation)

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
N = 1000
group1, group2 = capture_ttest(N, picoscope=True, splot=splot)

# Uncomment to overwrite
#npsave("picoscope_shuntmeasurement_tvla_group1.npy", group1)
#npsave("picoscope_shuntmeasurement_tvla_group2.npy", group2)

#### Analysis (HW not required)

In [ ]:
group1, group2, N = nptload("picoscope_shuntmeasurement")

In [ ]:
mean1 = np.mean(group1, axis=0)
mean2 = np.mean(group2, axis=0)
cw.plot(mean2) * cw.plot(mean1)

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]

plot_t(t_val, N, "SAM4S 120 MHz, Shunt Measurement (PicoScope)")

### With just the Clock

Doing this requires connecting the clock directly to channel A. The range will be adjusted to deal with the larger signal (you may want to adjust the range first before connecting the clock)

In [ ]:
scope.io.hs2 = None

In [ ]:
ps.setChannel(channel='A', coupling='AC', VRange=5, VOffset=0.0, enabled=True, BWLimited=0, probeAttenuation=1.0)

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
N = 20000
group1, group2 = capture_ttest(N, picoscope=True, splot=splot)

#Uncomment to overwrite
#np.save("picoscope_rawrxclock_tvla_group1_20k.npy", group1)
#np.save("picoscope_rawrxclock_tvla_group2_20k.npy", group2)

In [ ]:
#Two datasets were captured - this one @ 1.25 GS/s, 20K per group
group1, group2, N = nptload("picoscope_rawrxclock_20k")

In [ ]:
#This is a smaller dataset, 5K per group
group1, group2, N = nptload("picoscope_rawrxclock.npy")

In [ ]:
import numpy as np
mean1 = np.mean(group1, axis=0)
mean2 = np.mean(group2, axis=0)
cw.plot(mean2) * cw.plot(mean1)

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]

In [ ]:
plot_t(t_val, N, "Raw Clock Input")

#### Converting raw clock to time stamps

In [ ]:
group1time = create_diffs(group1)
group2time = create_diffs(group2)

In [ ]:
trim_length = min((min([len(g) for g in group1time]), min([len(g) for g in group2time])))
group1time = [g[0:trim_length] for g in group1time]
group2time = [g[0:trim_length] for g in group2time]

In [ ]:
import numpy as np
mean1 = np.mean(group1time, axis=0)
mean2 = np.mean(group2time, axis=0)
cw.plot(mean2) * cw.plot(mean1)

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1time, group2time, axis=0, equal_var=False)[0]

In [ ]:
plot_t(t_val, N, "Clock Input, Zero-Crossing to Jitter")

In [ ]:
#ps.close()

### With Zero Crossing Location Only Recorded (smaller traces)

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
N = 50000
group1, group2 = capture_ttest(N, picoscope=True, splot=splot, t

#Uncomment to overwrite
np.save("picoscope_rawrxclock_tvla_group1_zcross_50k.npy", group1)
np.save("picoscope_rawrxclock_tvla_group2_zcross_50k.npy", group2)

In [ ]:
group1, group2, N = nptload("picoscope_rawrxclock_zcross_50k")

In [ ]:
group1time = create_diffs_from_crossings(group1)
group2time = create_diffs_from_crossings(group2)

In [ ]:
trim_length = min((min([len(g) for g in group1time]), min([len(g) for g in group2time])))
group1time = [g[0:trim_length] for g in group1time]
group2time = [g[0:trim_length] for g in group2time]

In [ ]:
import numpy as np
mean1 = np.mean(group1time, axis=0)
mean2 = np.mean(group2time, axis=0)
cw.plot(mean2) * cw.plot(mean1)

In [ ]:
trim_length = min([len(g) for g in group1])
group1trim = [g[0:trim_length] for g in group1]

In [ ]:
#cw.plot(np.std(group1trim, axis=0))
cw.plot(np.std(group1time, axis=0))

In [ ]:
from scipy.stats import ttest_ind
t_val = ttest_ind(group1time, group2time, axis=0, equal_var=False)[0]

In [ ]:
plot_t(t_val, N, "Clock Input, Zero-Crossing to Jitter")

#### Comparing to Shunt Measurement

In [ ]:
g1s, g2s, Ns = nptload("picoscope_shuntmeasurement")

mean_trace_shunt = np.mean(g1s, axis=0)
mean_trace_clock = np.mean(group1time, axis=0)

In [ ]:
import matplotlib.pylab as plt
import scipy as sp

resampled = sp.signal.resample(mean_trace_shunt, int(len(mean_trace_shunt)*10.9))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

dt = 1 / (1.25E9 / 10.9) * 1000

time = np.arange(0, dt*5100, dt)

fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Shunt & Time Measurement Comparison (1.25 GS/s)')
ax1.plot(time, sp.signal.decimate(resampled, 100)[150:5250])
ax2.plot(time, mean_trace_clock[150:5250])

ax1.set_ylabel("Avg. Voltage")
ax2.set_ylabel("Avg. Cycle Delta")

plt.xlabel("Time (uS)")

ax1.grid(True)
ax2.grid(True)

#fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
1250 / 120